# 📊 Day 1 실습 2: 데이터 품질 검증 및 점검

## 학습 목표
- 전처리된 데이터의 품질 검증
- 토큰 길이 사전 점검
- RAFT 데이터 구조 검증
- 중복 및 이상치 탐지
- 데이터 분포 분석 및 시각화

## 시간: 14:00–14:40 (40분)

In [ ]:
# 필요한 라이브러리 확인 (01번에서 이미 설치되었으므로 확인만)
import importlib

def check_package(package_name, import_name=None):
    """패키지 설치 여부만 확인"""
    if import_name is None:
        import_name = package_name.replace('-', '_')
    
    try:
        importlib.import_module(import_name)
        print(f"✅ {package_name} 사용 가능")
        return True
    except ImportError:
        print(f"❌ {package_name} 설치 필요 - 01번 노트북을 먼저 실행하세요")
        return False

print("🚀 Day 1 실습 2: 데이터 품질 검증")
print("🔍 필요한 라이브러리 확인 중...")

# 02번에서 사용할 라이브러리들 확인
packages = [
    ("transformers", "transformers"),
    ("torch", "torch"), 
    ("datasets", "datasets"),
    ("jsonlines", "jsonlines"),
    ("pandas", "pandas"),
    ("numpy", "numpy"),
    ("matplotlib", "matplotlib"),
    ("seaborn", "seaborn"),
    ("tqdm", "tqdm"),
    ("scikit-learn", "sklearn")
]

all_available = True
print("📋 라이브러리 상태:")
for package_name, import_name in packages:
    if not check_package(package_name, import_name):
        all_available = False

if all_available:
    print("\n🎉 모든 라이브러리 준비 완료!")
    print("💡 데이터 품질 검증을 시작합니다.")
else:
    print("\n⚠️ 일부 라이브러리가 설치되지 않았습니다.")
    print("💡 먼저 01_data_preprocessing_and_validation.ipynb를 실행하세요.")

In [ ]:
import os
import json
import jsonlines
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any, Optional, Tuple
from tqdm import tqdm
import warnings
from datasets import Dataset
from transformers import AutoTokenizer, TrainerCallback
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib) - 데이터 품질 검증 차트에서 한글이 깨지지 않도록 설정
print("🔧 한글 폰트 설정 중...")
!apt-get update -qq
!apt-get install fonts-nanum -qq > /dev/null

import matplotlib.font_manager as fm

# 나눔바른고딕 폰트 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# 폰트 매니저에 폰트 추가 - 품질 분석 그래프에서 한글 표시를 위해 필요
fm.fontManager.addfont(fontpath)

# matplotlib 설정 업데이트 - 모든 품질 분석 차트에서 한글이 정상적으로 표시됨
plt.rcParams.update({
    'font.family': 'NanumBarunGothic',  # 기본 폰트를 나눔바른고딕으로 설정
    'axes.unicode_minus': False         # 음수 기호 표시 문제 해결 (통계 차트에서 중요)
})

# 시각화 스타일 설정 - 더 깔끔하고 전문적인 차트를 위한 설정
plt.style.use('default')  # 기본 스타일 사용
sns.set_palette("husl")   # 색상 팔레트 설정 - 구별하기 쉬운 색상 사용

print("✅ 한글 폰트 설정 완료 - 데이터 품질 분석 차트에서 한글이 정상 표시됩니다")
print("📦 라이브러리 import 완료!")

In [ ]:
# 추가 라이브러리 import (품질 분석에 필요한 특수 라이브러리들)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter, defaultdict
import re
from sentence_transformers import SentenceTransformer
from wordcloud import WordCloud

print("📦 추가 분석 라이브러리 import 완료!")

## 2. 전처리된 데이터 로드

In [ ]:
def load_processed_data():
    """
    전처리된 데이터와 메타데이터를 로드하는 함수
    
    Returns:
        train_data, valid_data, metadata
    """
    print("🔄 전처리된 데이터 로드 중...")
    
    
    # Train 데이터 로드
    train_data = []
    with jsonlines.open("processed_data/train_raft_ko.jsonl", "r") as reader:
        train_data = list(reader)
    
    # Valid 데이터 로드
    valid_data = []
    with jsonlines.open("processed_data/valid_raft_ko.jsonl", "r") as reader:
        valid_data = list(reader)
    
    # 메타데이터 로드
    with open("processed_data/metadata.json", "r", encoding="utf-8") as f:
        metadata = json.load(f)
    
    print(f"✅ 데이터 로드 완료:")
    print(f"  - Train: {len(train_data)}개 샘플")
    print(f"  - Valid: {len(valid_data)}개 샘플")
    
    return train_data, valid_data, metadata
        
    
# 데이터 로드
train_data, valid_data, metadata = load_processed_data()

if train_data is not None:
    print(f"\n📋 메타데이터 정보:")
    for key, value in metadata.items():
        print(f"  {key}: {value}")

## 3. 기본 데이터 품질 검증

### 📋 검증 항목
1. 데이터 무결성 확인
2. 필수 필드 존재 여부
3. 데이터 타입 검증
4. 빈 값 및 결측치 탐지

In [ ]:
def validate_data_integrity(data: List[Dict], data_name: str) -> Dict[str, Any]:
    """
    데이터 무결성 검증 함수
    
    Args:
        data: 검증할 데이터 리스트
        data_name: 데이터셋 이름
        
    Returns:
        검증 결과 딕셔너리
    """
    print(f"🔍 {data_name} 데이터 무결성 검증 중...")
    
    integrity_report = {
        "total_samples": len(data),
        "required_fields": ["messages", "type", "original_question", "original_answer"],
        "missing_fields": defaultdict(int),
        "empty_values": defaultdict(int),
        "invalid_types": defaultdict(int),
        "message_structure_errors": 0,
        "type_distribution": defaultdict(int)
    }
    
    for i, item in enumerate(data):
        # 1. 필수 필드 존재 확인
        for field in integrity_report["required_fields"]:
            if field not in item:
                integrity_report["missing_fields"][field] += 1
        
        # 2. 빈 값 확인
        for field in ["original_question", "original_answer"]:
            if field in item and (not item[field] or item[field].strip() == ""):
                integrity_report["empty_values"][field] += 1
        
        # 3. messages 구조 검증
        if "messages" in item:
            if not isinstance(item["messages"], list) or len(item["messages"]) != 3:
                integrity_report["message_structure_errors"] += 1
            else:
                # 메시지 역할 확인
                expected_roles = ["system", "user", "assistant"]
                actual_roles = [msg.get("role", "") for msg in item["messages"]]
                if actual_roles != expected_roles:
                    integrity_report["message_structure_errors"] += 1
        
        # 4. type 분포 확인
        if "type" in item:
            integrity_report["type_distribution"][item["type"]] += 1
    
    return integrity_report

def print_integrity_report(report: Dict[str, Any], data_name: str):
    """
    무결성 검증 결과 출력 함수
    """
    print(f"\n📊 {data_name} 무결성 검증 결과:")
    print(f"  총 샘플 수: {report['total_samples']}개")
    
    # 결측 필드
    if report['missing_fields']:
        print(f"  ❌ 결측 필드:")
        for field, count in report['missing_fields'].items():
            print(f"    {field}: {count}개 ({count/report['total_samples']:.1%})")
    else:
        print(f"  ✅ 필수 필드 모두 존재")
    
    # 빈 값
    if report['empty_values']:
        print(f"  ⚠️ 빈 값:")
        for field, count in report['empty_values'].items():
            print(f"    {field}: {count}개 ({count/report['total_samples']:.1%})")
    else:
        print(f"  ✅ 빈 값 없음")
    
    # 메시지 구조 오류
    if report['message_structure_errors'] > 0:
        error_rate = report['message_structure_errors'] / report['total_samples']
        print(f"  ❌ 메시지 구조 오류: {report['message_structure_errors']}개 ({error_rate:.1%})")
    else:
        print(f"  ✅ 메시지 구조 정상")
    
    # 타입 분포
    print(f"  📈 타입 분포:")
    for type_name, count in report['type_distribution'].items():
        print(f"    {type_name}: {count}개 ({count/report['total_samples']:.1%})")

# Train/Valid 데이터 무결성 검증
if train_data is not None:
    train_report = validate_data_integrity(train_data, "Train")
    valid_report = validate_data_integrity(valid_data, "Valid")
    
    print_integrity_report(train_report, "Train")
    print_integrity_report(valid_report, "Valid")

## 4. 토큰 길이 분석

### 🔍 분석 내용
1. 전체 대화 토큰 길이 분포
2. 메시지별 토큰 길이 분석
3. 4096 토큰 제한 준수 확인
4. Outlier 탐지

In [ ]:
# EXAONE 토크나이저 로드
print("🔄 토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct")
print("✅ 토크나이저 로드 완료")

def analyze_token_distribution(data: List[Dict], data_name: str) -> Dict[str, Any]:
    """
    토큰 길이 분포 분석 함수
    
    Args:
        data: 분석할 데이터
        data_name: 데이터셋 이름
        
    Returns:
        토큰 분석 결과
    """
    print(f"📊 {data_name} 토큰 길이 분석 중...")
    
    token_analysis = {
        "total_tokens": [],
        "system_tokens": [],
        "user_tokens": [],
        "assistant_tokens": [],
        "overflow_samples": [],
        "type_wise_tokens": {"positive": [], "negative": []}
    }
    
    for i, item in enumerate(tqdm(data, desc=f"{data_name} 토큰 분석")):
        if "messages" not in item or len(item["messages"]) != 3:
            continue
        
        messages = item["messages"]
        
        # 전체 대화 토큰 수
        full_conversation = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=False
        )
        total_tokens = len(tokenizer.encode(full_conversation))
        token_analysis["total_tokens"].append(total_tokens)
        
        # 메시지별 토큰 수
        for j, (msg, token_list) in enumerate(zip(messages, 
                                                 ["system_tokens", "user_tokens", "assistant_tokens"])):
            msg_tokens = len(tokenizer.encode(msg["content"]))
            token_analysis[token_list].append(msg_tokens)
        
        # 4096 토큰 초과 샘플 기록
        if total_tokens > 4096:
            token_analysis["overflow_samples"].append({
                "index": i,
                "total_tokens": total_tokens,
                "type": item.get("type", "unknown")
            })
        
        # Type별 토큰 분포
        sample_type = item.get("type", "unknown")
        if sample_type in token_analysis["type_wise_tokens"]:
            token_analysis["type_wise_tokens"][sample_type].append(total_tokens)
    
    return token_analysis

# 토큰 분석 실행
if train_data is not None:
    train_tokens = analyze_token_distribution(train_data, "Train")
    valid_tokens = analyze_token_distribution(valid_data, "Valid")
    
    # 결과 요약 출력
    def print_token_summary(token_analysis: Dict, data_name: str):
        total_tokens = token_analysis["total_tokens"]
        if not total_tokens:
            return
            
        print(f"\n📊 {data_name} 토큰 분석 결과:")
        print(f"  평균 토큰 수: {np.mean(total_tokens):.1f}")
        print(f"  중간값: {np.median(total_tokens):.1f}")
        print(f"  표준편차: {np.std(total_tokens):.1f}")
        print(f"  최소/최대: {min(total_tokens)} / {max(total_tokens)}")
        print(f"  4096 초과: {len(token_analysis['overflow_samples'])}개 ({len(token_analysis['overflow_samples'])/len(total_tokens):.1%})")
        
        # Type별 평균
        for type_name, tokens in token_analysis["type_wise_tokens"].items():
            if tokens:
                print(f"  {type_name} 평균: {np.mean(tokens):.1f} 토큰")
    
    print_token_summary(train_tokens, "Train")
    print_token_summary(valid_tokens, "Valid")

## 5. 데이터 분포 시각화

### 📈 시각화 차트
1. 토큰 길이 분포 히스토그램
2. Type별 토큰 길이 비교
3. 메시지 역할별 토큰 분포
4. Train/Valid 분포 비교

In [ ]:
def create_comprehensive_visualization(train_tokens: Dict, valid_tokens: Dict):
    """
    종합적인 데이터 품질 시각화 함수
    이 함수는 6개의 다른 관점에서 데이터를 분석하고 시각화합니다.
    각 차트는 데이터의 특정 측면을 보여주며, 전체적인 품질을 평가할 수 있게 해줍니다.
    """
    print("📊 종합 데이터 품질 대시보드 생성 중...")
    
    # 대시보드 스타일 서브플롯 생성 (3행 2열 구성)
    # 📊 의미: 6개의 차트로 데이터의 다양한 측면을 한번에 분석
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            "📏 토큰 길이 분포 비교 (Train vs Valid)",
            "🎯 Type별 토큰 길이 특성", 
            "💬 메시지 역할별 토큰 분포",
            "⚠️ 4096 토큰 초과 샘플 분석",
            "📦 토큰 길이 통계 (박스플롯)",
            "📈 누적 분포 함수 (CDF) 비교"
        ],
        specs=[
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]
        ]
    )
    
    # 1. 토큰 길이 분포 히스토그램 (Train vs Valid)
    # 📊 의미: Train과 Valid 데이터의 토큰 길이 분포가 유사한지 확인
    # - 두 분포가 비슷하면 데이터 분할이 잘 되었음을 의미
    # - 큰 차이가 있다면 분할 전략을 재검토해야 함
    fig.add_trace(
        go.Histogram(x=train_tokens["total_tokens"], name="Train 데이터", 
                    opacity=0.7, nbinsx=50, marker_color='lightblue'),
        row=1, col=1
    )
    fig.add_trace(
        go.Histogram(x=valid_tokens["total_tokens"], name="Valid 데이터", 
                    opacity=0.7, nbinsx=50, marker_color='lightcoral'),
        row=1, col=1
    )
    
    # 2. Type별 토큰 길이 분포
    # 📊 의미: RAFT의 Positive/Negative 샘플이 토큰 길이 측면에서 균형잡혀 있는지 확인
    # - Positive 샘플: 정답 context가 포함되어 보통 더 길 수 있음
    # - Negative 샘플: 정답 없는 distractor만 있어 비교적 짧을 수 있음
    colors = ['lightgreen', 'lightsalmon']
    for i, (type_name, tokens) in enumerate(train_tokens["type_wise_tokens"].items()):
        if tokens:
            fig.add_trace(
                go.Histogram(x=tokens, name=f"Train-{type_name.capitalize()}", 
                           opacity=0.7, nbinsx=30, marker_color=colors[i % len(colors)]),
                row=1, col=2
            )
    
    # 3. 메시지 역할별 토큰 분포
    # 📊 의미: System, User, Assistant 메시지의 길이 분포를 비교
    # - System: 보통 고정된 길이 (역할 설명)
    # - User: Context와 질문 포함으로 가장 길 수 있음  
    # - Assistant: 답변 길이로 적절한 범위에 있어야 함
    roles = ["system_tokens", "user_tokens", "assistant_tokens"] 
    role_names = ["System (역할 설명)", "User (질문+Context)", "Assistant (답변)"]
    role_colors = ['lightsteelblue', 'lightpink', 'lightgreen']
    
    for role, name, color in zip(roles, role_names, role_colors):
        if train_tokens[role]:
            fig.add_trace(
                go.Box(y=train_tokens[role], name=name, 
                      marker_color=color, boxmean=True),
                row=2, col=1
            )
    
    # 4. 4096 토큰 초과 분석
    # 📊 의미: 모델의 최대 입력 길이를 초과하는 샘플들의 Type별 분포
    # - 초과 샘플이 많으면 데이터 전처리나 Context 길이 조정 필요
    # - Type별로 초과 비율이 다르면 해당 Type의 구조적 문제 가능성
    overflow_types = defaultdict(int)
    for sample in train_tokens["overflow_samples"]:
        overflow_types[sample["type"]] += 1
    
    if overflow_types:
        fig.add_trace(
            go.Bar(x=list(overflow_types.keys()), y=list(overflow_types.values()),
                   name="토큰 초과 샘플", marker_color='red', opacity=0.7),
            row=2, col=2
        )
        # 초과율 표시를 위한 텍스트 추가
        total_samples = len(train_tokens["total_tokens"])
        for type_name, count in overflow_types.items():
            fig.add_annotation(
                x=type_name, y=count + 0.1,
                text=f"{count/total_samples:.1%}",
                showarrow=False, row=2, col=2
            )
    
    # 5. 토큰 길이 박스플롯 (Train vs Valid)
    # 📊 의미: 사분위수와 이상값을 통한 상세 분포 비교
    # - 중앙값, 사분위수로 분포의 중심과 퍼짐 정도 파악
    # - 이상값(outlier)으로 비정상적으로 긴 샘플 식별
    fig.add_trace(
        go.Box(y=train_tokens["total_tokens"], name="Train 분포",
               marker_color='lightblue', boxmean=True),
        row=3, col=1
    )
    fig.add_trace(
        go.Box(y=valid_tokens["total_tokens"], name="Valid 분포", 
               marker_color='lightcoral', boxmean=True),
        row=3, col=1
    )
    
    # 6. 누적 분포 함수 (CDF)
    # 📊 의미: 특정 토큰 길이 이하의 샘플 비율을 보여줌
    # - 예: 2000 토큰 이하 샘플이 전체의 몇 %인지 확인
    # - 데이터의 분포 특성과 길이 제한 설정에 도움
    train_sorted = np.sort(train_tokens["total_tokens"])
    train_cdf = np.arange(1, len(train_sorted) + 1) / len(train_sorted)
    
    valid_sorted = np.sort(valid_tokens["total_tokens"])  
    valid_cdf = np.arange(1, len(valid_sorted) + 1) / len(valid_sorted)
    
    fig.add_trace(
        go.Scatter(x=train_sorted, y=train_cdf, mode='lines', 
                   name="Train CDF", line=dict(color='blue', width=2)),
        row=3, col=2
    )
    fig.add_trace(
        go.Scatter(x=valid_sorted, y=valid_cdf, mode='lines',
                   name="Valid CDF", line=dict(color='red', width=2)),
        row=3, col=2
    )
    
    # 4096 토큰 제한선 추가 (중요한 기준선)
    # 📊 의미: 모델의 최대 입력 길이 표시로 데이터 적합성 판단
    fig.add_vline(x=4096, line_dash="dash", line_color="orange", 
                  annotation_text="🚨 모델 최대 길이: 4096 토큰", 
                  annotation_position="top", row=3, col=2)
    
    # 평균선도 추가 (참고용)
    train_mean = np.mean(train_tokens["total_tokens"])
    fig.add_vline(x=train_mean, line_dash="dot", line_color="green",
                  annotation_text=f"📊 Train 평균: {train_mean:.0f}토큰",
                  annotation_position="bottom", row=3, col=2)
    
    # 전체 레이아웃 업데이트
    fig.update_layout(
        height=1200,  # 충분한 높이로 각 차트가 잘 보이도록
        title_text="📊 데이터 품질 종합 분석 대시보드<br><sub>파인튜닝 전 필수 점검 사항</sub>",
        showlegend=True,
        title_x=0.5,
        title_font_size=18
    )
    
    # 각 서브플롯별 축 제목 설정
    fig.update_xaxes(title_text="토큰 길이", row=1, col=1)
    fig.update_yaxes(title_text="샘플 수", row=1, col=1)
    
    fig.update_xaxes(title_text="토큰 길이", row=1, col=2)
    fig.update_yaxes(title_text="샘플 수", row=1, col=2)
    
    fig.update_yaxes(title_text="토큰 길이", row=2, col=1)
    
    fig.update_xaxes(title_text="샘플 Type", row=2, col=2)
    fig.update_yaxes(title_text="초과 샘플 수", row=2, col=2)
    
    fig.update_yaxes(title_text="토큰 길이", row=3, col=1)
    
    fig.update_xaxes(title_text="토큰 길이", row=3, col=2)
    fig.update_yaxes(title_text="누적 확률", row=3, col=2)
    
    return fig

# 시각화 생성 및 표시
if train_data is not None and train_tokens["total_tokens"]:
    print("🎨 데이터 품질 대시보드 생성 중...")
    dashboard_fig = create_comprehensive_visualization(train_tokens, valid_tokens)
    dashboard_fig.show()
    
    # HTML로도 저장하여 상호작용 가능하게 함
    dashboard_fig.write_html("processed_data/interactive_dashboard.html")
    print("✅ 상호작용 대시보드 저장: processed_data/interactive_dashboard.html")
    print("\n🔍 대시보드 해석 가이드:")
    print("  📏 토큰 길이 분포: Train/Valid 분포가 유사해야 분할이 적절함")
    print("  🎯 Type별 분포: Positive/Negative 샘플의 토큰 길이 균형 확인")
    print("  💬 메시지 역할별: System < Assistant < User 순으로 길이가 일반적")
    print("  ⚠️ 토큰 초과: 4096을 초과하는 샘플은 잘리거나 제거 필요")  
    print("  📦 박스플롯: 이상값(점들)이 많으면 데이터 정제 필요")
    print("  📈 CDF: 90% 샘플이 4096 이하에 있어야 이상적")
    print("\n💡 이 대시보드를 통해 파인튜닝 전 데이터 적합성을 종합 판단할 수 있습니다!")

## 6. 중복 및 유사도 분석

### 🔍 분석 내용
1. 정확한 텍스트 중복 탐지
2. 의미적 유사도 분석
3. Question 다양성 검증
4. Answer 품질 분석

In [ ]:
def analyze_duplicates_and_similarity(data: List[Dict], sample_size: int = 100) -> Dict[str, Any]:
    """
    중복 및 유사도 분석 함수
    
    Args:
        data: 분석할 데이터
        sample_size: 유사도 분석을 위한 샘플 크기
        
    Returns:
        중복/유사도 분석 결과
    """
    print("🔍 중복 및 유사도 분석 중...")
    
    analysis_result = {
        "exact_duplicates": {
            "questions": 0,
            "answers": 0,
            "full_conversations": 0
        },
        "question_diversity": {},
        "answer_diversity": {},
        "semantic_similarity": {}
    }
    
    # 텍스트 추출
    questions = [item.get("original_question", "") for item in data]
    answers = [item.get("original_answer", "") for item in data]
    
    # 1. 정확한 중복 탐지
    unique_questions = set(questions)
    unique_answers = set(answers)
    
    analysis_result["exact_duplicates"]["questions"] = len(questions) - len(unique_questions)
    analysis_result["exact_duplicates"]["answers"] = len(answers) - len(unique_answers)
    
    # 2. Question/Answer 다양성 분석
    analysis_result["question_diversity"] = {
        "total_questions": len(questions),
        "unique_questions": len(unique_questions),
        "diversity_ratio": len(unique_questions) / len(questions) if questions else 0,
        "avg_length": np.mean([len(q) for q in questions if q]),
        "length_std": np.std([len(q) for q in questions if q])
    }
    
    analysis_result["answer_diversity"] = {
        "total_answers": len(answers),
        "unique_answers": len(unique_answers),
        "diversity_ratio": len(unique_answers) / len(answers) if answers else 0,
        "avg_length": np.mean([len(a) for a in answers if a]),
        "length_std": np.std([len(a) for a in answers if a])
    }
    
    # 3. 의미적 유사도 분석 (샘플링)
    if len(data) > sample_size:
        sampled_indices = np.random.choice(len(data), sample_size, replace=False)
        sampled_questions = [questions[i] for i in sampled_indices if questions[i]]
        sampled_answers = [answers[i] for i in sampled_indices if answers[i]]
    else:
        sampled_questions = [q for q in questions if q]
        sampled_answers = [a for a in answers if a]
    
    try:
        print("🔄 의미적 유사도 계산 중... (시간이 걸릴 수 있습니다)")
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        
        if sampled_questions:
            question_embeddings = model.encode(sampled_questions[:50])  # 계산 시간 단축
            q_similarity_matrix = cosine_similarity(question_embeddings)
            
            # 대각선 제외한 유사도 점수
            q_similarities = q_similarity_matrix[np.triu_indices_from(q_similarity_matrix, k=1)]
            
            analysis_result["semantic_similarity"]["questions"] = {
                "mean_similarity": np.mean(q_similarities),
                "max_similarity": np.max(q_similarities),
                "high_similarity_pairs": np.sum(q_similarities > 0.8)
            }
        
        if sampled_answers:
            answer_embeddings = model.encode(sampled_answers[:50])
            a_similarity_matrix = cosine_similarity(answer_embeddings)
            
            a_similarities = a_similarity_matrix[np.triu_indices_from(a_similarity_matrix, k=1)]
            
            analysis_result["semantic_similarity"]["answers"] = {
                "mean_similarity": np.mean(a_similarities),
                "max_similarity": np.max(a_similarities),
                "high_similarity_pairs": np.sum(a_similarities > 0.8)
            }
            
        print("✅ 의미적 유사도 계산 완료")
        
    except Exception as e:
        print(f"⚠️ 의미적 유사도 계산 실패: {e}")
        analysis_result["semantic_similarity"] = {"error": str(e)}
    
    return analysis_result

# 중복/유사도 분석 실행
if train_data is not None:
    similarity_analysis = analyze_duplicates_and_similarity(train_data)
    
    # 결과 출력
    print("\n📊 중복 및 유사도 분석 결과:")
    print(f"\n🔍 정확한 중복:")
    for key, value in similarity_analysis["exact_duplicates"].items():
        print(f"  {key}: {value}개")
    
    print(f"\n📝 Question 다양성:")
    q_div = similarity_analysis["question_diversity"]
    print(f"  전체/고유: {q_div['total_questions']}/{q_div['unique_questions']}")
    print(f"  다양성 비율: {q_div['diversity_ratio']:.1%}")
    print(f"  평균 길이: {q_div['avg_length']:.1f}자")
    
    print(f"\n💬 Answer 다양성:")
    a_div = similarity_analysis["answer_diversity"]
    print(f"  전체/고유: {a_div['total_answers']}/{a_div['unique_answers']}")
    print(f"  다양성 비율: {a_div['diversity_ratio']:.1%}")
    print(f"  평균 길이: {a_div['avg_length']:.1f}자")
    
    if "error" not in similarity_analysis["semantic_similarity"]:
        print(f"\n🧠 의미적 유사도:")
        if "questions" in similarity_analysis["semantic_similarity"]:
            q_sim = similarity_analysis["semantic_similarity"]["questions"]
            print(f"  Question 평균 유사도: {q_sim['mean_similarity']:.3f}")
            print(f"  Question 최대 유사도: {q_sim['max_similarity']:.3f}")
            print(f"  고유사도(>0.8) 쌍: {q_sim['high_similarity_pairs']}개")

## 7. RAFT 구조 검증

### 🎯 RAFT 특화 검증
1. Positive/Negative 샘플 균형
2. Context 개수 분포
3. Context 관련성 분석
4. Distractor 품질 평가

In [ ]:
def analyze_raft_structure(data: List[Dict]) -> Dict[str, Any]:
    """
    RAFT 데이터 구조 분석 함수
    
    Args:
        data: RAFT 구조 데이터
        
    Returns:
        RAFT 구조 분석 결과
    """
    print("🎯 RAFT 구조 분석 중...")
    
    raft_analysis = {
        "type_distribution": defaultdict(int),
        "context_analysis": {
            "context_counts": [],
            "context_lengths": []
        },
        "message_analysis": {
            "user_message_lengths": [],
            "system_message_lengths": [],
            "assistant_message_lengths": []
        },
        "structure_issues": []
    }
    
    for i, item in enumerate(data):
        # 1. Type 분포
        item_type = item.get("type", "unknown")
        raft_analysis["type_distribution"][item_type] += 1
        
        # 2. Message 구조 분석
        if "messages" in item and len(item["messages"]) == 3:
            messages = item["messages"]
            
            # 각 메시지 길이 수집
            raft_analysis["message_analysis"]["system_message_lengths"].append(
                len(messages[0].get("content", ""))
            )
            raft_analysis["message_analysis"]["user_message_lengths"].append(
                len(messages[1].get("content", ""))
            )
            raft_analysis["message_analysis"]["assistant_message_lengths"].append(
                len(messages[2].get("content", ""))
            )
            
            # 3. User 메시지에서 context 정보 추출
            user_content = messages[1].get("content", "")
            
            # Context 개수 추정 (간단한 휴리스틱)
            context_matches = re.findall(r'컨텍스트 \d+:', user_content)
            context_count = len(context_matches)
            
            if context_count > 0:
                raft_analysis["context_analysis"]["context_counts"].append(context_count)
            
            # Context 섹션 길이
            context_section_match = re.search(r'=== 컨텍스트 ===\n(.*?)\n=== 질문 ===', 
                                             user_content, re.DOTALL)
            if context_section_match:
                context_length = len(context_section_match.group(1).strip())
                raft_analysis["context_analysis"]["context_lengths"].append(context_length)
        else:
            raft_analysis["structure_issues"].append(f"Sample {i}: Invalid message structure")
    
    return raft_analysis

def visualize_raft_analysis(raft_analysis: Dict[str, Any]):
    """
    RAFT 분석 결과 시각화
    """
    # 한글 폰트 설정 재확인 - RAFT 차트에서 한글이 깨지지 않도록 보장
    plt.rcParams.update({
        'font.family': 'NanumBarunGothic',
        'axes.unicode_minus': False
    })
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Type 분포 파이차트
    type_counts = list(raft_analysis["type_distribution"].values())
    type_labels = list(raft_analysis["type_distribution"].keys())
    
    ax1.pie(type_counts, labels=type_labels, autopct='%1.1f%%', startangle=90)
    ax1.set_title('RAFT Type 분포', fontsize=14, fontweight='bold')
    
    # 2. Context 개수 분포
    context_counts = raft_analysis["context_analysis"]["context_counts"]
    if context_counts:
        ax2.hist(context_counts, bins=range(1, max(context_counts)+2), 
                alpha=0.7, edgecolor='black')
        ax2.set_xlabel('Context 개수')
        ax2.set_ylabel('샘플 수')
        ax2.set_title('Context 개수별 분포', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
    
    # 3. 메시지 길이 분포
    msg_analysis = raft_analysis["message_analysis"]
    roles = ['system_message_lengths', 'user_message_lengths', 'assistant_message_lengths']
    role_names = ['System', 'User', 'Assistant']
    
    box_data = [msg_analysis[role] for role in roles if msg_analysis[role]]
    box_labels = [name for role, name in zip(roles, role_names) if msg_analysis[role]]
    
    if box_data:
        ax3.boxplot(box_data, labels=box_labels)
        ax3.set_ylabel('메시지 길이 (문자)')
        ax3.set_title('메시지 역할별 길이 분포', fontsize=14, fontweight='bold')
        ax3.grid(True, alpha=0.3)
    
    # 4. Context 길이 분포
    context_lengths = raft_analysis["context_analysis"]["context_lengths"]
    if context_lengths:
        ax4.hist(context_lengths, bins=30, alpha=0.7, color='lightblue', edgecolor='black')
        ax4.set_xlabel('Context 섹션 길이 (문자)')
        ax4.set_ylabel('빈도')
        ax4.set_title('Context 길이 분포', fontsize=14, fontweight='bold')
        ax4.axvline(np.mean(context_lengths), color='red', linestyle='--', 
                   label=f'평균: {np.mean(context_lengths):.0f}')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('processed_data/raft_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

# RAFT 구조 분석 실행
if train_data is not None:
    raft_analysis = analyze_raft_structure(train_data)
    
    # 분석 결과 출력
    print("\n🎯 RAFT 구조 분석 결과:")
    print(f"\n📊 Type 분포:")
    total_samples = sum(raft_analysis["type_distribution"].values())
    for type_name, count in raft_analysis["type_distribution"].items():
        percentage = (count / total_samples) * 100 if total_samples > 0 else 0
        print(f"  {type_name}: {count}개 ({percentage:.1f}%)")
    
    # Context 분석
    context_counts = raft_analysis["context_analysis"]["context_counts"]
    context_lengths = raft_analysis["context_analysis"]["context_lengths"]
    
    if context_counts:
        print(f"\n📋 Context 분석:")
        print(f"  평균 Context 개수: {np.mean(context_counts):.1f}개")
        print(f"  Context 개수 범위: {min(context_counts)} ~ {max(context_counts)}개")
        
        # Context 개수별 빈도
        context_freq = Counter(context_counts)
        print(f"  Context 개수별 분포:")
        for count, freq in sorted(context_freq.items()):
            print(f"    {count}개: {freq}회 ({freq/len(context_counts):.1%})")
    
    if context_lengths:
        print(f"\n📏 Context 길이 통계:")
        print(f"  평균 길이: {np.mean(context_lengths):.0f}자")
        print(f"  중간값: {np.median(context_lengths):.0f}자")
        print(f"  길이 범위: {min(context_lengths)} ~ {max(context_lengths)}자")
    
    # 구조 문제 확인
    if raft_analysis["structure_issues"]:
        print(f"\n⚠️ 구조 문제 발견:")
        for issue in raft_analysis["structure_issues"][:5]:  # 처음 5개만 표시
            print(f"  {issue}")
        if len(raft_analysis["structure_issues"]) > 5:
            print(f"  ... 총 {len(raft_analysis['structure_issues'])}개 문제")
    else:
        print(f"\n✅ 구조 문제 없음")
    
    # 시각화
    raft_viz = visualize_raft_analysis(raft_analysis)
    print("\n✅ RAFT 분석 차트 저장: processed_data/raft_analysis.png")

## 8. 최종 품질 점수 및 권장사항

### 📋 데이터 품질 스코어카드
1. 무결성 점수 (0-100)
2. 다양성 점수 (0-100) 
3. 구조 적합성 점수 (0-100)
4. 토큰 효율성 점수 (0-100)
5. 종합 품질 점수

In [ ]:
def calculate_quality_score(train_report: Dict, valid_report: Dict, 
                          train_tokens: Dict, similarity_analysis: Dict,
                          raft_analysis: Dict) -> Dict[str, Any]:
    """
    데이터 품질 종합 점수 계산
    
    Returns:
        품질 점수 딕셔너리
    """
    print("📊 데이터 품질 종합 점수 계산 중...")
    
    quality_scores = {}
    
    # 1. 무결성 점수 (0-100)
    integrity_issues = (
        sum(train_report.get("missing_fields", {}).values()) +
        sum(train_report.get("empty_values", {}).values()) +
        train_report.get("message_structure_errors", 0)
    )
    total_train_samples = train_report.get("total_samples", 1)
    integrity_score = max(0, 100 - (integrity_issues / total_train_samples) * 100)
    quality_scores["integrity_score"] = integrity_score
    
    # 2. 다양성 점수 (0-100)
    q_diversity = similarity_analysis.get("question_diversity", {}).get("diversity_ratio", 0)
    a_diversity = similarity_analysis.get("answer_diversity", {}).get("diversity_ratio", 0)
    diversity_score = ((q_diversity + a_diversity) / 2) * 100
    quality_scores["diversity_score"] = diversity_score
    
    # 3. 구조 적합성 점수 (0-100)
    structure_issues = len(raft_analysis.get("structure_issues", []))
    structure_score = max(0, 100 - (structure_issues / total_train_samples) * 100)
    quality_scores["structure_score"] = structure_score
    
    # 4. 토큰 효율성 점수 (0-100)
    overflow_rate = len(train_tokens.get("overflow_samples", [])) / len(train_tokens.get("total_tokens", [1]))
    token_efficiency_score = max(0, 100 - (overflow_rate * 100))
    quality_scores["token_efficiency_score"] = token_efficiency_score
    
    # 5. RAFT 균형 점수 (0-100)
    type_dist = raft_analysis.get("type_distribution", {})
    positive_ratio = type_dist.get("positive", 0) / sum(type_dist.values()) if type_dist else 0
    # 이상적인 비율(0.6)에서 얼마나 벗어났는지 계산
    balance_deviation = abs(positive_ratio - 0.6)
    raft_balance_score = max(0, 100 - (balance_deviation * 200))  # 편차를 점수로 변환
    quality_scores["raft_balance_score"] = raft_balance_score
    
    # 6. 종합 품질 점수 (가중평균)
    weights = {
        "integrity_score": 0.3,
        "diversity_score": 0.25,
        "structure_score": 0.2,
        "token_efficiency_score": 0.15,
        "raft_balance_score": 0.1
    }
    
    overall_score = sum(quality_scores[key] * weight for key, weight in weights.items())
    quality_scores["overall_score"] = overall_score
    
    return quality_scores

def generate_recommendations(quality_scores: Dict[str, float], 
                           train_tokens: Dict, 
                           similarity_analysis: Dict) -> List[str]:
    """
    품질 점수 기반 권장사항 생성
    """
    recommendations = []
    
    # 무결성 관련
    if quality_scores["integrity_score"] < 90:
        recommendations.append(
            "⚠️ 데이터 무결성 개선 필요: 결측값이나 구조적 문제가 있는 샘플을 수정하세요."
        )
    
    # 다양성 관련
    if quality_scores["diversity_score"] < 80:
        recommendations.append(
            "📝 데이터 다양성 개선 필요: 중복된 질문이나 답변을 제거하고 더 다양한 샘플을 추가하세요."
        )
    
    # 토큰 효율성 관련
    if quality_scores["token_efficiency_score"] < 90:
        overflow_count = len(train_tokens.get("overflow_samples", []))
        recommendations.append(
            f"📏 토큰 길이 최적화 필요: {overflow_count}개 샘플이 4096 토큰을 초과합니다. "
            "긴 context를 줄이거나 분할을 고려하세요."
        )
    
    # RAFT 균형 관련
    if quality_scores["raft_balance_score"] < 85:
        recommendations.append(
            "⚖️ RAFT 샘플 균형 조정 필요: Positive와 Negative 샘플 비율을 6:4로 맞추세요."
        )
    
    # 의미적 유사도 관련
    if "semantic_similarity" in similarity_analysis:
        q_sim = similarity_analysis["semantic_similarity"].get("questions", {})
        if q_sim.get("high_similarity_pairs", 0) > 5:
            recommendations.append(
                "🧠 의미적 중복 제거 필요: 유사한 질문들이 많이 발견되었습니다. "
                "의미적으로 중복되는 샘플을 제거하세요."
            )
    
    # 전체 점수가 낮은 경우
    if quality_scores["overall_score"] < 75:
        recommendations.append(
            "🔄 전면적인 데이터 개선 필요: 전체 품질 점수가 낮습니다. "
            "데이터 수집부터 전처리까지 전 과정을 재검토하세요."
        )
    elif quality_scores["overall_score"] >= 90:
        recommendations.append(
            "✅ 우수한 데이터 품질: 현재 데이터는 파인튜닝에 적합한 품질을 가지고 있습니다."
        )
    
    if not recommendations:
        recommendations.append(
            "✅ 전반적으로 양호한 데이터 품질입니다. 파인튜닝을 진행해도 좋습니다."
        )
    
    return recommendations

def create_quality_scorecard_viz(quality_scores: Dict[str, float]):
    """
    품질 스코어카드 시각화
    """
    # 점수 데이터 준비
    score_names = [
        "무결성", "다양성", "구조 적합성", 
        "토큰 효율성", "RAFT 균형", "종합 점수"
    ]
    score_keys = [
        "integrity_score", "diversity_score", "structure_score",
        "token_efficiency_score", "raft_balance_score", "overall_score"
    ]
    scores = [quality_scores[key] for key in score_keys]
    
    # 색상 매핑 (점수에 따라)
    def get_color(score):
        if score >= 90:
            return 'green'
        elif score >= 75:
            return 'orange'
        else:
            return 'red'
    
    colors = [get_color(score) for score in scores]
    
    # 바 차트 생성
    fig, ax = plt.subplots(figsize=(14, 8))
    bars = ax.barh(score_names, scores, color=colors, alpha=0.7)
    
    # 점수 표시
    for i, (bar, score) in enumerate(zip(bars, scores)):
        ax.text(score + 1, i, f'{score:.1f}', va='center', fontweight='bold')
    
    # 차트 설정
    ax.set_xlim(0, 105)
    ax.set_xlabel('점수', fontsize=12, fontweight='bold')
    ax.set_title('📊 데이터 품질 스코어카드', fontsize=16, fontweight='bold', pad=20)
    
    # 점수 구간 표시
    ax.axvline(x=75, color='orange', linestyle='--', alpha=0.5, label='양호 기준 (75점)')
    ax.axvline(x=90, color='green', linestyle='--', alpha=0.5, label='우수 기준 (90점)')
    ax.legend()
    
    # 그리드
    ax.grid(True, axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('processed_data/quality_scorecard.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

# 품질 점수 계산 및 권장사항 생성
if train_data is not None:
    quality_scores = calculate_quality_score(
        train_report, valid_report, train_tokens, similarity_analysis, raft_analysis
    )
    
    recommendations = generate_recommendations(
        quality_scores, train_tokens, similarity_analysis
    )
    
    # 결과 출력
    print("\n🏆 데이터 품질 종합 평가 결과:")
    print("=" * 50)
    
    score_names = {
        "integrity_score": "무결성 점수",
        "diversity_score": "다양성 점수", 
        "structure_score": "구조 적합성 점수",
        "token_efficiency_score": "토큰 효율성 점수",
        "raft_balance_score": "RAFT 균형 점수",
        "overall_score": "📊 종합 품질 점수"
    }
    
    for key, name in score_names.items():
        score = quality_scores[key]
        if score >= 90:
            status = "🟢 우수"
        elif score >= 75:
            status = "🟡 양호"
        else:
            status = "🔴 개선 필요"
        
        print(f"{name}: {score:.1f}점 {status}")
    
    print(f"\n💡 권장사항:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
    
    # 스코어카드 시각화
    scorecard_fig = create_quality_scorecard_viz(quality_scores)
    print("\n✅ 품질 스코어카드 저장: processed_data/quality_scorecard.png")
    
    # 품질 리포트 JSON 저장
    quality_report = {
        "quality_scores": quality_scores,
        "recommendations": recommendations,
        "analysis_timestamp": pd.Timestamp.now().isoformat(),
        "data_summary": {
            "train_samples": len(train_data),
            "valid_samples": len(valid_data),
            "avg_token_length": np.mean(train_tokens["total_tokens"]) if train_tokens["total_tokens"] else 0
        }
    }
    
    with open("processed_data/quality_report.json", "w", encoding="utf-8") as f:
        json.dump(quality_report, f, ensure_ascii=False, indent=2, default=str)
    
    print("✅ 품질 리포트 저장: processed_data/quality_report.json")

## 9. 최종 요약 및 다음 단계

### ✅ 완료된 검증 항목
1. **데이터 무결성 검증**: 필수 필드, 빈 값, 구조 오류 확인
2. **토큰 길이 분석**: 분포, 초과 샘플, 효율성 평가
3. **중복 및 유사도 분석**: 정확한 중복, 의미적 유사도 측정
4. **RAFT 구조 검증**: Type 균형, Context 품질, 구조 적합성
5. **종합 품질 평가**: 5개 영역 점수 + 전체 점수
6. **시각화 및 리포트**: 대시보드, 차트, JSON 리포트

### 📁 생성된 파일들
- `processed_data/token_analysis_dashboard.png`: 토큰 분석 대시보드
- `processed_data/raft_analysis.png`: RAFT 구조 분석 차트
- `processed_data/quality_scorecard.png`: 품질 스코어카드
- `processed_data/quality_report.json`: 종합 품질 리포트

### 🔄 다음 단계
**03_fine_tuning_with_lora.ipynb**에서 실제 파인튜닝을 진행합니다!

In [ ]:
print("🎯 Day 1 실습 2 완료!")
print("=" * 60)

if train_data is not None:
    final_summary = {
        "데이터 현황": {
            "Train 샘플": len(train_data),
            "Valid 샘플": len(valid_data),
            "평균 토큰 길이": f"{np.mean(train_tokens['total_tokens']):.1f}" if train_tokens['total_tokens'] else "N/A",
            "4096 토큰 초과율": f"{len(train_tokens.get('overflow_samples', [])) / len(train_tokens.get('total_tokens', [1])):.1%}" if train_tokens.get('total_tokens') else "N/A"
        },
        "RAFT 구조": {
            "Positive 샘플": raft_analysis.get("type_distribution", {}).get("positive", 0),
            "Negative 샘플": raft_analysis.get("type_distribution", {}).get("negative", 0),
            "평균 Context 개수": f"{np.mean(raft_analysis.get('context_analysis', {}).get('context_counts', [4])):.1f}개" if raft_analysis.get('context_analysis', {}).get('context_counts') else "4개"
        },
        "품질 점수": {
            "종합 점수": f"{quality_scores['overall_score']:.1f}점",
            "무결성": f"{quality_scores['integrity_score']:.1f}점",
            "다양성": f"{quality_scores['diversity_score']:.1f}점"
        }
    }
    
    print("📊 최종 요약:")
    for category, items in final_summary.items():
        print(f"\n{category}:")
        for key, value in items.items():
            print(f"  {key}: {value}")
    
    print(f"\n📁 생성된 파일:")
    files = [
        "token_analysis_dashboard.png",
        "raft_analysis.png", 
        "quality_scorecard.png",
        "quality_report.json"
    ]
    for file in files:
        print(f"  - processed_data/{file}")
    
    print(f"\n🚀 다음 단계: 03_fine_tuning_with_lora.ipynb에서 파인튜닝을 시작하세요!")
    
    # 종합 품질 등급 판정
    overall_score = quality_scores['overall_score']
    if overall_score >= 90:
        grade = "A (우수)"
        message = "데이터 품질이 우수합니다. 파인튜닝을 진행하세요! 🌟"
    elif overall_score >= 75:
        grade = "B (양호)"
        message = "데이터 품질이 양호합니다. 파인튜닝을 진행해도 좋습니다. ✅"
    elif overall_score >= 60:
        grade = "C (보통)"
        message = "데이터 품질이 보통입니다. 권장사항을 검토 후 진행하세요. ⚠️"
    else:
        grade = "D (개선 필요)"
        message = "데이터 품질 개선이 필요합니다. 권장사항을 적용하세요. 🔄"
    
    print(f"\n🏆 최종 품질 등급: {grade}")
    print(f"💬 {message}")
    
else:
    print("❌ 데이터를 불러올 수 없어 분석을 완료하지 못했습니다.")
    print("💡 먼저 01_data_preprocessing_and_validation.ipynb를 실행하세요.")